# FPL API Data Pulls
This Jupyter notebook handles all data pull requests for my project analyzing data from the Fantasy Premier League (FPL) API. It utilizes the requests, JSON, and pandas libraries to parse JSON files into pandas dataframes and then CSVs.

Here are the endpoints utilized in the code for reference:

+ *Root URL* - https://fantasy.premierleague.com/api/
+ *Gameweeks, Teams, Positions, and List of All Players* - https://fantasy.premierleague.com/api/bootstrap-static/
+ *League Standings* - https://fantasy.premierleague.com/api/leagues-classic/{league_id}/standings?page_standings={page_number}
+ *Weekly Manager History Data* - https://fantasy.premierleague.com/api/entry/{manager_id}/history/
+ *Weekly Player History Data* - https://fantasy.premierleague.com/api/element-summary/{element_id}/

This first block of code sets the root url for the API as fpl_api_url and generates a CSV for all gameweek info in addition to a series of completed gameweeks -- this comes in handy when pulling data in the middle of the season and conducting analysis up to that point thus far.

In [7]:
import requests, json
import pandas as pd

fpl_api_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(fpl_api_url+'bootstrap-static/').json()

# generate panda dataframe
gameweeks = pd.json_normalize(r['events'])
gameweeks.to_csv('gameweek_info.csv', index=False)

# generate series of completed gameweek ids
completed_gameweeks = gameweeks.loc[gameweeks['finished'] == True, 'id']
gameweeks.head()

,id,name,deadline_time,release_time,average_entry_score,finished,data_checked,highest_scoring_entry,deadline_time_epoch,deadline_time_game_offset,...,most_transferred_in,top_element,transfers_made,most_captained,most_vice_captained,overrides.element_types,overrides.pick_multiplier,top_element_info.id,top_element_info.points,top_element_info
0,1,Gameweek 1,2024-08-16T17:30:00Z,None,57,True,True,3546234.0,1723829400,0,...,27.0,328.0,0,351.0,351.0,[],None,328.0,14.0,NaN
1,2,Gameweek 2,2024-08-24T10:00:00Z,None,69,True,True,9442126.0,1724493600,0,...,594.0,177.0,13364453,351.0,401.0,[],None,177.0,20.0,NaN
2,3,Gameweek 3,2024-08-31T10:00:00Z,None,64,True,True,430195.0,1725098400,0,...,177.0,328.0,23723836,351.0,328.0,[],None,328.0,17.0,NaN
3,4,Gameweek 4,2024-09-14T10:00:00Z,None,51,True,True,3560750.0,1726308000,0,...,129.0,185.0,25930392,351.0,328.0,[],None,185.0,15.0,NaN
4,5,Gameweek 5,2024-09-21T10:00:00Z,None,58,True,True,1773336.0,1726912800,0,...,58.0,180.0,13036058,351.0,351.0,[],None,180.0,16.0,NaN


## Auxiliary Functions
These functions are helper functions only called within the context of other functions. Here's a quick rundown of each's purpose:

**check_request**

This function accepts a json_request_url and uses try/excepts to check for a valid JSON response. This function is used later to make sure our data requests are succesful before trying to parse and compile them into dataframes/CSVs.

**pull_manager_history_keys**

This function is used to set the columns for blank dataframes created within other functions more directly related to creating dataframes. It takes the FPL API url and a dataframe containing manager ID's (singular identifying information for managers playing the game) and returns the keys utilized in the JSON response that will become the blank dataframe's column labels.

**create_empty_dicts**

This function is used to help generate blank rows for missing manager data.

In [9]:
def check_request(json_request_url):

    error_message = ''
    
    # send request to API
    try:
        json_request = requests.get(json_request_url)

    #catch timeouts
    except requests.exceptions.ReadTimeout:
        error_message = 'Timeout'
        return error_message

    #catch connnection refusals from host
    except requests.exceptions.ConnectionError:
        error_message = 'Connection Refused'
        return error_message
    
    #catch empty responses, non-json responses, or other response code errors
    if(
        json_request.status_code != 200 and 
        json_request.headers["content-type"].strip().startswith("application/json")
    ):
        error_message = f'response error: {json_request.status_code}'
        return error_message

    return True

In [11]:
# find the first record with data and record the keys for the data dict
def pull_manager_history_keys(fpl_api_url, manager_df):

    # iterate over list of manager IDs to find a valid JSON response containing the keys we're looking for 
    for index_name in manager_df.index:
        manager_id = manager_df.loc[index_name, 'id']
        manager_history_json = requests.get(fpl_api_url + 'entry/' + str(manager_id) + '/history/').json()

        # if the desired keys are found, break the loop and return the keys object
        if 'current' in manager_history_json:
            manager_history_keys = manager_history_json['current'][0].keys()
            return manager_history_keys
            
    else:
        return False
# takes a list of keys and returns an empty dictionary with the key value pairs inside    
def create_empty_dicts(dict_keys, manager_id, missing_weeks_list):
    empty_list = []
    for i in range(len(missing_weeks_list)):
        empty_dict = dict.fromkeys(dict_keys)
        empty_dict['manager_id'] = manager_id
        empty_dict['event'] = missing_weeks_list[i]
        empty_list.append(empty_dict)
    return empty_list

## Data Pull Functions

**pull_manager_standings**

This function is used to compile a table of current standings for a manager league for up to the number of managers specified in the num_managers parameter. On the API, the results are broken up by pages each containing records for 50 managers. The remainder and page number parameters of the function allows for us to keep track of where we leave off when only a portion of page's records are pulled. This ensures we don't skip over any records if doing successive pulls. Any errors encountered in requesting the data are compiled and returned in error_list.

In [13]:
def pull_manager_standings(league_id, num_managers, base_url, page_number, remainder=None):
    
    league_url_standings = f'{base_url}leagues-classic/{league_id}/standings?page_standings='
    error_list = []
    df_list = []
    
    # check to see if the function has been passed a dataframe containing unused rows from a previous pull
    if remainder is None or remainder.empty:
        num_rows = 0

    # if unused rows are present, add them to the list first and iterate num_rows by the number of rows present in remainder df
    else:
        df_list = df_list + remainder.to_dict('records')
        num_rows = len(remainder)
    
    # use while loop to iterate page by page until data for the desired number of managers has been pulled
    while num_rows < num_managers:
        
        request_url = league_url_standings + str(page_number)

        # check to see if request returns valid json
        # if not, return error message and move to the next page
        connection_test = check_request(request_url)

        if connection_test == True:
            page_request = requests.get(request_url)
            page_response = page_request.json()
            page_dict = page_response['standings']['results']
            df_list = df_list + page_dict
            num_rows += len(page_dict)
            page_number +=1
        else:
            error_list.append(connection_test)
            page_number +=1

    #concatenate df_list into single manager_overall_df
    manager_league_df = pd.DataFrame(df_list)

    # this section checks the final data frame to make sure the number of managers in the df matches the requested number
    # if number exceeds, remove required number of rows from the tail and store removed rows in new_remainder_df
    if len(manager_league_df) != num_managers:
        num_rows_to_remove = len(manager_league_df) - num_managers
        new_remainder_df = manager_league_df.tail(num_rows_to_remove)
        manager_league_df.drop(manager_league_df.tail(num_rows_to_remove).index,
            inplace = True)

    else:
        new_remainder_df = pd.DataFrame()
        
    # keep track of last page in case you want to add more managers from later pages
    # to resume from where you left off, you would call pull_manager_standings with last_page as page_number
    last_page = page_number
    
    return manager_league_df, error_list, last_page, new_remainder_df

**pull_manager_history**

This function pulls the week-by-week history for a manager, utilizing their unique manager ID. It returns a tuple with this information as a dataframe along with a list of gameweeks for which the manager has data present. Sometimes, a manager's ID will return the following JSON response:  {"detail": "No Entry matches the given query."}. This is usually caught as a 404 error by connection_test but an extra check is included here for safety. Manager_ID and gameweek are appended to each row to make concatenating multiple manager data pulls into a single dataframe easier.

In [15]:
def pull_manager_history(url, manager_id, keyword):

    #initialize empty error_message dict
    error_message = {'manager_id': manager_id, 'error':'',}
    request_url = f'{url}entry/{manager_id}/history/'

    # check to see if request returns valid json
    connection_test = check_request(request_url)

    # only proceed to parsing JSON response if connection test passes
    # Otherwise, return the error encountered in the connection test as a dictionary
    if connection_test != True:
        error_message['manager_id'] = manager_id
        error_message['error'] = connection_test
        return error_message
    
    else:
        manager_history_json = requests.get(request_url).json()

        #check if returned json response does not contain history data (i.e. no 'current' key)
        #these errors are usually caught as 404 errors in check_request but extra check added just to be safe
        if keyword not in manager_history_json:
            error_message['error'] = 'No Entry matches the given query.'
            return error_message
    
        #if code has made it here, manager history is safe to pull
        manager_history_list = manager_history_json[keyword]
        gameweeks_present = []
        
        #add in manager_id
        for entry in manager_history_list:
            entry['manager_id'] = manager_id 
            gameweeks_present.append(entry['event'])
        return manager_history_list, gameweeks_present

These next two functions utilize the manager ID's of a league's standings (from pull_manager_history) to aggregate weekly manager data for multiple managers into a single dataframe. 

**Method One: compile_league_history_weekly_no_replacements**

This first method will only aggregate data for the managers present in the dataframe passed into the function. If any number of weeks of data is missing for this manager, they are represented by blanks (even if no weekly data is present for *any* gameweek).

**Method Two: compile_league_history_weekly_with_replacement**

This second method is similar to the first in that it aggregates weekly data for multiple managers and will insert blank rows if a manager is missing data for a specific gameweek. However, it departs from the first by skipping over managers who do not have *any* gameweek data present (i.e. 404 error). It will then pull a new list of managers from the API to fill out any remaining spots not filled in the initial list of managers. The managers who were excluded can be tracked using the error_message_df returned by the function.

In this way, method two ensures you will always have data for a set number of managers (regardless of league position) while method one ensures the data will always be limited to whatever can be found for a set list of managers (keeping the dataset limited to managers in the top *n* of the standings).

In [17]:
# method one -- generate week by week only for manager_ids present in manager_df with 
# missing people are represented by blank rows for each gameweek
# each manager_id in the original_df will be present in the returned df
def compile_league_history_weekly_no_replacements(manager_df, completed_gameweeks_list, manager_history_keys, fpl_api_url):

    manager_history_total = []
    error_messages = []
    
    for manager_id in manager_df['id']:

        web_pull = pull_manager_history(fpl_api_url, manager_id, 'current')
            
        # check to see if pull_manager_history returned an error dictionary
        # if yes, add the dictionary to the error_messages list and add blank rows for each completed gameweek for the manager
        if isinstance(web_pull, dict):
                
            error_messages.append(web_pull)
            empty_list = create_empty_dicts(manager_history_keys, manager_id, completed_gameweeks_list)
            manager_history_total = manager_history_total + empty_list
    
        else:
            
            manager_history_dict_list = web_pull[0]
            
            # check to which gameweeks are present in manager pull data
            # add in blank rows for any missing gameweeks
            gameweeks_present = web_pull[1]
            gameweeks_absent = list(set(completed_gameweeks_list) - set(gameweeks_present))
            
            if len(gameweeks_absent) > 0:
                empty_gameweeks_list = create_empty_dicts(manager_history_keys, manager_id, gameweeks_absent)
                manager_history_dict_list = manager_history_dict_list + empty_gameweeks_list
                manager_history_total = manager_history_total + manager_history_dict_list
            else:
                manager_history_total = manager_history_total + manager_history_dict_list
    
    manager_history_df = pd.DataFrame(manager_history_total)
    error_message_df = pd.DataFrame(error_messages)

    return manager_history_df, error_message_df

In [125]:
# method two -- generate week by week for a set of manager_ids, replacing managers who have no data with new managers who have data
# returned df will not have all manager_ids from the original df present, as these are replaced by new managers with data
# removed manager_ids are tracked through the error_message df
# useful when you want a specific sample size of data as opposed to whatever data exists for the top X number of managers
def compile_league_history_weekly_with_replacements(manager_df, completed_gameweeks_list, manager_history_keys, remainder_df, page_number, fpl_api_url):

    error_messages = []
    sample_size = manager_df['id'].nunique()
    # used to store all manager data even through multiple iterations of the for loop
    manager_history_running_total_df = pd.DataFrame()
    last_page = page_number
    current_remainder = remainder_df
    
    while True:

        # initialize list to store manager data in
        # resets with each for loop iteration
        manager_history_for_loop_total = []
        
        for manager_id in manager_df['id']:
        
            web_pull = pull_manager_history(fpl_api_url, manager_id, 'current')
        
            # check to see if pull_manager_history returned an error dictionary
            # if yes, add the dictionary to the error_messages list and add blank rows for each completed gameweek for the manager
            if isinstance(web_pull, dict):
                    
                error_messages.append(web_pull)
        
            else:
                manager_history_dict_list = web_pull[0]
                
                # check to which gameweeks are present in manager pull data
                # add in blank rows for any missing gameweeks
                gameweeks_present = web_pull[1]
                gameweeks_absent = list(set(completed_gameweeks_list) - set(gameweeks_present))
                
                if len(gameweeks_absent) > 0:
                    empty_gameweeks_list = create_empty_dicts(manager_history_keys, manager_id, gameweeks_absent)
                    manager_history_dict_list = manager_history_dict_list + empty_gameweeks_list
                    manager_history_for_loop_total = manager_history_for_loop_total + manager_history_dict_list
                else:
                    manager_history_for_loop_total = manager_history_for_loop_total + manager_history_dict_list
        
        if manager_history_running_total_df.empty:
            manager_history_running_total_df = pd.DataFrame(manager_history_for_loop_total)
        else:
            manager_history_for_loop_df = pd.DataFrame(manager_history_for_loop_total)
            manager_history_running_total_df = pd.concat([manager_history_running_total_df, manager_history_for_loop_df], ignore_index=True)
       
        error_message_df = pd.DataFrame(error_messages)
        
        print(f'{manager_history_running_total_df['manager_id'].nunique()} managers in the dataframe.')
        
        # checks to see if we have the total number of desired managers in the dataframe
        if manager_history_running_total_df['manager_id'].nunique() == sample_size:
            return manager_history_running_total_df, error_message_df

        replacements_needed = sample_size - manager_history_running_total_df['manager_id'].nunique()
        print(f'Pulling {replacements_needed} additional row(s).')
        manager_df_new, error_list, page_number_new, remainder_new = pull_manager_standings(league_id, replacements_needed, fpl_api_url, last_page, remainder=current_remainder)
    
        #set variables to new values ahead of next while loop iteration
        manager_df = manager_df_new
        last_page = page_number_new
        current_remainder = remainder_new

## Generate CSVs
The following few blocks of code create CSVs for the top 1000 managers in the overall league standings, the aggregated weekly histories of these top 1000 managers, and then the aggregated weekly histories of the top 1000 managers with data present in the API.

In [87]:
# variables for data pulls
# redefined here for reference
fpl_api_url = 'https://fantasy.premierleague.com/api/'
num_managers = 1000
league_id = 314
completed_gameweeks_list = completed_gameweeks.to_list()

In [129]:
# first CSV -- top 1000 managers in overall league
page_number = 1
manager_league_df, error_list, last_page, new_remainder_df = pull_manager_standings(league_id, num_managers, fpl_api_url, page_number, remainder=None)
manager_league_df.to_csv('overall_league_manager_standings_top_1000.csv')
if len(error_list) == 0:
    print('No errors in data pull')
else:
    print(error_list)
manager_league_df.head()

No errors in data pull


,id,event_total,player_name,rank,last_rank,rank_sort,total,entry,entry_name,has_played
0,1450135,61,Lovro Budišin,1,1,1,2752,235307,Aina Krafth Bree,True
1,10448238,72,Jack Brennan,2,4,2,2712,1620838,Hakuna Mateta,True
2,66708,55,Keyuran Govender,3,3,3,2702,12901,The Palmer Trees,True
3,7888534,65,Miguel Piccand,4,6,4,2700,1236816,@miguelpiccand,True
4,32633580,49,Max Littleproud,5,2,5,2697,4538028,Levenshulme Utd,True


In [27]:
# csv weekly histories with no replacements
manager_df = pd.read_csv('overall_league_manager_standings_top_1000.csv')
manager_history_keys = pull_manager_history_keys(fpl_api_url, manager_df)
manager_history_df, error_message_df = compile_league_history_weekly_no_replacements(manager_df, completed_gameweeks_list, manager_history_keys, fpl_api_url)
manager_history_df.to_csv('overall_league_week_by_week_top_1000_no_replacements.csv')
error_message_df.to_csv('errors_top1K_week_by_week_no_replacements_data_pull.csv')
manager_history_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37000 entries, 0 to 36999
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   event                 37000 non-null  int64  
 1   points                23296 non-null  float64
 2   total_points          23296 non-null  float64
 3   rank                  23272 non-null  float64
 4   rank_sort             23272 non-null  float64
 5   overall_rank          23294 non-null  float64
 6   percentile_rank       23272 non-null  float64
 7   bank                  23296 non-null  float64
 8   value                 23296 non-null  float64
 9   event_transfers       23296 non-null  float64
 10  event_transfers_cost  23296 non-null  float64
 11  points_on_bench       23296 non-null  float64
 12  manager_id            37000 non-null  int64  
dtypes: float64(11), int64(2)
memory usage: 3.7 MB


In [131]:
# csv weekly histories WITH replacements
# taken from top 1000 managers in overall league pull
page_number = last_page
remainder_df = new_remainder_df
manager_df = pd.read_csv('overall_league_manager_standings_top_1000.csv')
manager_history_keys = pull_manager_history_keys(fpl_api_url, manager_df)
manager_history_with_replacements_df, error_message_df = compile_league_history_weekly_with_replacements(manager_df, completed_gameweeks_list, manager_history_keys, remainder_df, page_number, fpl_api_url)
manager_history_with_replacements_df.to_csv('overall_league_week_by_week_top_1000_WITH_replacements.csv')
error_message_df.to_csv('errors_top1K_weekly_WITH_replacements_data_pull.csv')
manager_history_with_replacements_df.info()

639 managers in the dataframe.
Pulling 361 additional row(s).
870 managers in the dataframe.
Pulling 130 additional row(s).
954 managers in the dataframe.
Pulling 46 additional row(s).
984 managers in the dataframe.
Pulling 16 additional row(s).
996 managers in the dataframe.
Pulling 4 additional row(s).
998 managers in the dataframe.
Pulling 2 additional row(s).
998 managers in the dataframe.
Pulling 2 additional row(s).
999 managers in the dataframe.
Pulling 1 additional row(s).
1000 managers in the dataframe.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37000 entries, 0 to 36999
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   event                 37000 non-null  int64  
 1   points                36491 non-null  float64
 2   total_points          36491 non-null  float64
 3   rank                  36366 non-null  float64
 4   rank_sort             36366 non-null  float64
 5   overall_ran

## Player Data
The first block of code aggregates a list of players who have played in the Premier League this season, merging in team and position data from other sections of the API. The second block takes this dataframe and aggregates weekly data for each player present into a single dataframe.

In [133]:
fpl_api_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(fpl_api_url+'bootstrap-static/').json()

# create players dataframe
players = pd.json_normalize(r['elements'])

# select columns of interest
players_ref = players[[
    'element_type',
    'first_name',
    'id',
    'points_per_game',
    'second_name',
    'selected_by_percent',
    'team',
    'team_code',
    'minutes'
]]
# filter out players who haven't played this season
players_ref = players_ref[players_ref['minutes'] > 0]

# create teams dataframe 
teams = pd.json_normalize(r['teams'])
teams = teams[['name', 'id']]

# get position information from 'element_types' field
positions = pd.json_normalize(r['element_types'])
positions = positions[['singular_name', 'id']]

# join players to teams
players_ref = pd.merge(
    left=players_ref,
    right=teams,
    left_on='team',
    right_on='id'
)

# join player positions
players_ref = players_ref.merge(
    positions,
    left_on='element_type',
    right_on='id'
)

# select columns of interest from merged dataframe
players_ref = players_ref[[
    'first_name',
    'second_name', 
    'id_x', 
    'name', 
    'singular_name' 
]]

# rename ambiguous columns
players_ref = players_ref.rename(columns={'id_x': 'player_id', 
                                          'name': 'team_name',
                                          'singular_name': 'position'})

#combine first_name and second_name columns to player_name
players_ref['player_name'] = players_ref[['first_name', 'second_name']].agg(' '.join, axis=1)
players_ref = players_ref.drop(columns=['first_name', 'second_name'])
col_order = ['player_name', 'player_id', 'team_name', 'position']
players_ref = players_ref.reindex(columns=col_order)
players_ref.head()

,player_name,player_id,team_name,position
0,Gabriel Fernando de Jesus,2,Arsenal,Forward
1,Gabriel dos Santos Magalhães,3,Arsenal,Defender
2,Kai Havertz,4,Arsenal,Forward
3,Jurriën Timber,6,Arsenal,Defender
4,Jorge Luiz Frello Filho,7,Arsenal,Midfielder


In [137]:
#pulls player week-by-week history -- modified version of pull_manager_history w/ updated endpoints
#don't need to fill in blanks as players don't play every week (whereas managers enter a team every week)
#if successful, returns player_history as a list of dicts
#returns error message if unsuccesful
def pull_player_history(url, player_id, keyword):

    #initialize empty error_message dict
    error_message = {'player_id': player_id, 'error':'',}
    request_url = f'{url}element-summary/{player_id}/'

    # check to see if request returns valid json
    connection_test = check_request(request_url)

    if connection_test != True:
        error_message['player_id'] = manager_id
        error_message['error'] = connection_test
        return error_message
    
    else:
        #json is safe to parse if code makes it here
        player_history_json = requests.get(request_url).json()
    
        #check if returned json response does not contain history data (i.e. no 'history' key)
        #these errors are usually caught above as 404 but extra check added just to be safe
        if keyword not in player_history_json:
            error_message['error'] = 'No Entry matches the given query.'
            return error_message
    
        #if code has made it here, player history is safe to pull
        player_history_list = player_history_json[keyword]
        
        #add in player_id
        for entry in player_history_list:
            entry['player_id'] = player_id 
        return player_history_list

#iterate over list of player_ids to pull data for each
player_history_total = []
player_error_messages = []
player_missing_data = []

for player_id in players_ref['player_id']:

    web_pull = pull_player_history(fpl_api_url, player_id, 'history')

    if isinstance(web_pull, dict):
        player_error_messages.append(web_pull)
    
    else:
        player_history_dict_list = web_pull
        player_history_total = player_history_total + player_history_dict_list

player_history_df = pd.DataFrame(player_history_total)
player_error_messages_df = pd.DataFrame(player_error_messages)

#only generate error csv if errors are present
if player_error_messages_df.empty:
    print('No errors in data pull.')
else:
    player_error_messages_df.to_csv('error_history.csv', index=False)

# merge players_ref with player_history_df
player_history_df = pd.merge(
    left=player_history_df,
    right=players_ref,
    left_on='player_id',
    right_on='player_id'
)

player_history_df.to_csv('player_weekly_history.csv')
player_history_df.head()

No errors in data pull.


,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,modified,...,mng_goals_scored,value,transfers_balance,selected,transfers_in,transfers_out,player_id,player_name,team_name,position
0,2,2,20,0,True,2024-08-17T14:00:00Z,2,0,1,False,...,0,70,0,199810,0,0,2,Gabriel Fernando de Jesus,Arsenal,Forward
1,2,11,2,0,False,2024-08-24T16:30:00Z,0,2,2,False,...,0,69,-53975,176166,12170,66145,2,Gabriel Fernando de Jesus,Arsenal,Forward
2,2,21,5,0,True,2024-08-31T11:30:00Z,1,1,3,False,...,0,68,-72583,106691,3238,75821,2,Gabriel Fernando de Jesus,Arsenal,Forward
3,2,39,18,1,False,2024-09-15T13:00:00Z,0,1,4,False,...,0,68,-27513,82577,1997,29510,2,Gabriel Fernando de Jesus,Arsenal,Forward
4,2,47,13,0,False,2024-09-22T15:30:00Z,2,2,5,False,...,0,68,1642,89599,7682,6040,2,Gabriel Fernando de Jesus,Arsenal,Forward
